In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

from pathlib import Path

# Read Simulated RAW image

In [ ]:
def readImageToArray(imagePath):
    """
    Read an image from a file and convert it to a numpy array.
    """
    image = Image.open(imagePath)
    return np.array(image)

def plotImages(images, titles=None, cols=3):
    """
    Plot a list of images in a grid.
    """
    n = len(images)
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    for i, ax in enumerate(axes.flat):
        if i < n:
            ax.imshow(images[i], cmap='gray')
            if titles is not None:
                ax.set_title(titles[i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def read_raw_with_metadata(filename):
    with open(filename, 'rb') as f:
        # Read metadata
        width = int(f.readline().decode().strip().split()[1])
        height = int(f.readline().decode().strip().split()[1])
        bayer_pattern = f.readline().decode().strip().split()[1]
        
        # Skip the "DATA" line
        f.readline()
        
        # Read raw pixel data
        raw_data = np.fromfile(f, dtype=np.uint8).reshape((height, width))
    
    return raw_data, width, height, bayer_pattern

# # Example usage
# raw_data, width, height, bayer_pattern = read_raw_with_metadata('butterfly.raw')
# print(f"Width: {width}, Height: {height}, Bayer Pattern: {bayer_pattern}")

In [ ]:
datapath = Path('demosaic/data')
path = Path('demosaic/results')

# Read the original image
original = readImageToArray(datapath / 'butterfly.jpg')
shape = original.shape

# Read raw image
raw, width, height, bayer_pattern = read_raw_with_metadata(path / 'butterfly.raw')
print(f"Width: {width}, Height: {height}, Bayer Pattern: {bayer_pattern}")

In [ ]:
# Convert Bayer image to RGB image (RGGB pattern)
rawRGB = np.zeros((shape[1], shape[0], 3), dtype=np.uint8)
rawRGB[0::2, 0::2, 0] = raw[0::2, 0::2]  # R
rawRGB[0::2, 1::2, 1] = raw[0::2, 1::2]  # G
rawRGB[1::2, 0::2, 1] = raw[1::2, 0::2]  # G
rawRGB[1::2, 1::2, 2] = raw[1::2, 1::2]  # B
rawRGB = np.clip(rawRGB, 0, 255).astype(np.uint8)

# Load Bayer RGB from C++ code
rawRGBCpp = readImageToArray(path / 'butterfly.raw_rgb.png')
rawRGBCpp = np.array(rawRGBCpp)

print(np.all(rawRGBCpp == rawRGB))

In [ ]:
plotImages([raw, rawRGB, rawRGBCpp], titles=['Raw', 'Bayer RGB', 'Bayer RGB from C++'])

# Read Real RAW Image

In [ ]:
import rawpy

In [ ]:
with rawpy.imread('demosaic/data/alpaca.RAF') as raw:
    rgb = raw.postprocess()
    print(rgb.shape)
    plt.imshow(rgb)
    plt.show()
    raw_image = raw.raw_image
    print(raw_image.shape)
    plt.imshow(raw_image, cmap='gray')
    plt.colorbar()
    plt.show()
    rawValue = raw_image.copy()

    bayer = raw.raw_colors.copy()

In [ ]:
bayer